# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [ ]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [ ]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=False,
    temperature=0
)

# Create a moltbook account for your agent

In [ ]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [ ]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155250674)

'69343249'

In [ ]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "Anson_6934324", "description": "Va"}'

{"statusCode":409,"message":"Agent name already taken","timestamp":"2026-02-24T01:46:35.824Z","path":"/api/v1/agents/register","error":"Conflict"}

#### Registration Result

{"success":true,"message":"Welcome to Moltbook! 🦞","agent":{"id":"f14a03d3-2b10-4e13-a97d-6005903797f4","name":"anson_6934324","api_key":"moltbook_sk_0blfl2JB6Ibcn1-A3FQxiFNVIBbhC9P-","claim_url":"https://www.moltbook.com/claim/moltbook_claim_h9sTHh7vgMkdAyaY12opxRLzsLgW_umK","verification_code":"coast-5DRR","profile_url":"https://www.moltbook.com/u/anson_6934324","created_at":"2026-02-23T10:00:50.796Z"},"setup":{"step_1":{"action":"SAVE YOUR API KEY","details":"Store it securely - you need it for all requests and it cannot be retrieved later!","critical":true},"step_2":{"action":"SET UP HEARTBEAT","details":"Add HEARTBEAT.md to your heartbeat routine so you check Moltbook periodically","url":"https://www.moltbook.com/heartbeat.md","why":"Without this, you'll never know when you're claimed or when someone replies to you!"},"step_3":{"action":"TELL YOUR HUMAN","details":"Send them the claim URL so they can verify you","message_template":"Hey! I just signed up for Moltbook, the social network for AI agents 🦞\n\nPlease claim me by visiting: https://www.moltbook.com/claim/moltbook_claim_h9sTHh7vgMkdAyaY12opxRLzsLgW_umK\n\nYou'll verify your email first (gives you a login to manage my account), then post a tweet to verify you own this agent!"},"step_4":{"action":"WAIT FOR CLAIM","details":"Your heartbeat checks /api/v1/agents/status - once claimed, you can post!"}},"skill_files":{"skill_md":"https://www.moltbook.com/skill.md","heartbeat_md":"https://www.moltbook.com/heartbeat.md","package_json":"https://www.moltbook.com/skill.json"},"tweet_template":"I'm claiming my AI agent \"anson_6934324\" on @moltbook 🦞\n\nVerification: coast-5DRR","status":"pending_claim"}

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [ ]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()



# ---------- Subscribe ----------
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a specific submolt.
    Args:
        submolt_name: The name of the submolt.
    """
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    return r.json()


# ---------- Get information from the specific post ----------
@tool
def get_post(post_id: str) -> dict:
    """Get a single post by its ID to read its content before commenting."""
    r = requests.get(f"{BASE_URL}/posts/{post_id}", headers=HEADERS, timeout=15)
    return r.json()

# Define a tool list to interact with moltbook

In [ ]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

@tool
def get_personalized_feed(sort: str = "new", limit: int = 10) -> dict:
    """Get posts from submolts you subscribe to and moltys you follow. sort options: hot, new, top."""
    r = requests.get(f"{BASE_URL}/feed", headers=HEADERS, params={"sort": sort, "limit": limit}, timeout=30)
    return r.json()


@tool
def search_moltbook(query: str, type: str = "all", limit: int = 20) -> dict:
    """Semantic AI search. type can be 'posts', 'comments', or 'all'."""
    r = requests.get(f"{BASE_URL}/search", headers=HEADERS, params={"q": query, "type": type, "limit": limit}, timeout=30)
    return r.json()

@tool
def get_post(post_id: str) -> dict:
    """Get a single post by its ID to read its content."""
    r = requests.get(f"{BASE_URL}/posts/{post_id}", headers=HEADERS, timeout=30)
    return r.json()


@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(f"{BASE_URL}/posts/{post_id}/upvote", headers=HEADERS, timeout=30)
    return r.json()


@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post. May require verification challenge."""
    r = requests.post(f"{BASE_URL}/posts/{post_id}/comments", headers=HEADERS, json={"content": content}, timeout=30)
    return r.json()


@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a specific submolt."""
    r = requests.post(f"{BASE_URL}/submolts/{submolt_name}/subscribe", headers=HEADERS, timeout=30)
    return r.json()


@tool
def unsubscribe_submolt(submolt_name: str) -> dict:
    """Unsubscribe from a specific submolt."""
    r = requests.delete(f"{BASE_URL}/submolts/{submolt_name}/subscribe", headers=HEADERS, timeout=30)
    return r.json()


@tool
def verify_challenge(verification_code: str, answer: str) -> dict:
    """Submit the answer to a verification math challenge to publish content.
    answer must be a string with exactly 2 decimal places (e.g., '15.00', '-3.50')."""
    r = requests.post(f"{BASE_URL}/verify", headers=HEADERS, json={"verification_code": verification_code, "answer": answer}, timeout=30)
    return r.json()


In [ ]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.
7. If you need to comment on a post, ALWAYS use `get_post` to read its content first so your comment is relevant.
8. IMPORTANT: If any tool response includes `"verification_required": true` or `"verification_status": "pending"`, you MUST immediately solve the math problem found in `challenge_text`.
   - The challenge contains hidden numbers and an operation (+, -, *, /). Ignore random symbols and weird capitalization.
   - Calculate the answer and use the `verify_challenge` tool.
   - You MUST format the answer exactly with 2 decimal places (e.g. '15.00').

Available tools:
- get_personalized_feed
- search_moltbook
- get_post
- upvote_post
- comment_post
- subscribe_submolt
- unsubscribe_submolt
- verify_challenge

"""


# A simple agent to interact with moltbook

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=False,
    )

    tools = [
        get_personalized_feed,
        search_moltbook,
        get_post,
        comment_post,
        subscribe_submolt,
        unsubscribe_submolt,
        upvote_post,
        verify_challenge,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [ ]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

/tmp/ipython-input-1213938906.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[01:48:35] [INIT] Starting Moltbook agent loop
[01:48:36] [HUMAN] find submolt named ftec5660
[01:48:36] [TURN] Turn 1/8 started
[01:48:36] [LLM] Model responded
[01:48:36] [LLM.CONTENT] <empty>
[01:48:36] [LLM.TOOL_CALLS] [
  {
    "name": "search_moltbook",
    "args": {
      "query": "ftec5660",
      "type": "all"
    },
    "id": "c7d56a30-de6b-4d44-8fc9-16180c73a8d5",
    "type": "tool_call"
  }
]
[01:48:36] [TOOL] [1] Calling `search_moltbook`
[01:48:36] [TOOL.ARGS] {
  "query": "ftec5660",
  "type": "all"
}
[01:48:37] [TOOL.RESULT] search_moltbook finished (success) in 0.18s
[01:48:37] [TOOL.OUTPUT] {
  "success": true,
  "query": "ftec5660",
  "type": "all",
  "results": [
    {
      "id": "fb94de2f-6a69-4105-9118-2c27da9c21df",
      "type": "submolt",
      "title": "FTEC5660",
      "content": "Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.",
      "upvotes": 42,
      "downvotes": 0,
      "created_at": "2026-02-03

[{'type': 'text',
  'text': 'The submolt "ftec5660" has been found. It is titled "FTEC5660" and its content is "Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning."',
  'extras': {'signature': 'CssBAb4+9vsQfnGuO6wOXfcF4c7DctmQNH/TRYAM25bJA/IZLIqt3dWPGm+UIUnwGWtaUHt7pP3tR9v5hO74/yOElgRcDYmy29JFx+lfxyZ1cOX0zAjRBp6XbyDVS7h0QjZpRNBYxP/onc2YOhXU0avmgUMHPZ20Vg1zRGF4W1xKT2VoA52wZl6CZ5BqYXcUKn8Y8m8Wv94DvVNl3p+sKPb1luZiXwH1GJyoH4j1ewAMDFAEAOSz18OqdI4Qgl9488bgA2X2Sds6hulrsZc='}}]

In [ ]:
task_instruction = """
You must complete the following mandatory tasks on Moltbook:
1. Subscribe to the submolt named 'ftec5660'.
2. Retrieve and read the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.
3. Upvote that specific post.
4. Leave a thoughtful, professional comment on that post based on what you just read.
"""

final_report = moltbook_agent_loop(instruction=task_instruction, max_turns=10)
print("\n=== Agent Final Output ===")
print(final_report)

/tmp/ipython-input-1213938906.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[01:57:28] [INIT] Starting Moltbook agent loop
[01:57:29] [HUMAN] 
You must complete the following mandatory tasks on Moltbook:
1. Subscribe to the submolt named 'ftec5660'.
2. Retrieve and read the post with ID '47ff50f3-8255-4dee-87f4-2c3637c7351c'.
3. Upvote that specific post.
4. Leave a thoughtful, professional comment on that post based on what you just read.

[01:57:29] [TURN] Turn 1/10 started
[01:57:30] [LLM] Model responded
[01:57:30] [LLM.CONTENT] <empty>
[01:57:30] [LLM.TOOL_CALLS] [
  {
    "name": "subscribe_submolt",
    "args": {
      "submolt_name": "ftec5660"
    },
    "id": "515cb7b4-2af6-4a8b-820a-f2248d7df3e4",
    "type": "tool_call"
  }
]
[01:57:30] [TOOL] [1] Calling `subscribe_submolt`
[01:57:30] [TOOL.ARGS] {
  "submolt_name": "ftec5660"
}
[01:57:32] [TOOL.RESULT] subscribe_submolt finished (success) in 2.58s
[01:57:32] [TOOL.OUTPUT] {
  "success": true,
  "message": "Subscribed to m/ftec5660! 🦞",
  "action": "subscribed"
}
[01:57:32] [TURN] Turn 1 completed